In [54]:
from dotenv import load_dotenv
import os
load_dotenv() 

True

In [55]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [56]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_taxi_data

from_year = 2023
# to_year = datetime.now().year
to_year = 2024
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_taxi_data(year)

    chunks.append(rides_one_year)
    break

# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2023 to 2024
File already exists for 2023-01.
Loading data for 2023-01...
Total records: 3,066,766
Valid records: 2,993,140
Records dropped: 73,626 (2.40%)
Successfully processed data for 2023-01.
File already exists for 2023-02.
Loading data for 2023-02...
Total records: 2,913,955
Valid records: 2,845,058
Records dropped: 68,897 (2.36%)
Successfully processed data for 2023-02.
File already exists for 2023-03.
Loading data for 2023-03...
Total records: 3,403,766
Valid records: 3,331,705
Records dropped: 72,061 (2.12%)
Successfully processed data for 2023-03.
File already exists for 2023-04.
Loading data for 2023-04...
Total records: 3,288,250
Valid records: 3,214,922
Records dropped: 73,328 (2.23%)
Successfully processed data for 2023-04.
File already exists for 2023-05.
Loading data for 2023-05...
Total records: 3,513,649
Valid records: 3,435,875
Records dropped: 77,774 (2.21%)
Successfully processed data for 2023-05.
File already exists for 2023-06.
Loading dat

In [57]:
rides

,pickup_datetime,pickup_location_id
0,2023-01-01 00:32:10,161
1,2023-01-01 00:55:08,43
2,2023-01-01 00:25:04,48
3,2023-01-01 00:03:48,138
4,2023-01-01 00:10:29,107
...,...,...
37463263,2023-12-31 23:04:34,233
37463264,2023-12-31 23:08:15,48
37463265,2023-12-31 23:16:15,196
37463266,2023-12-31 23:21:58,140


In [58]:
rides.shape

(37463268, 2)

In [59]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [60]:
ts_data.shape

(2277600, 3)

In [61]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277600 entries, 0 to 2277599
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int16         
 2   rides               int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 26.1 MB


In [62]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
...,...,...,...
2277595,2023-12-31 19:00:00,263,188
2277596,2023-12-31 20:00:00,263,198
2277597,2023-12-31 21:00:00,263,232
2277598,2023-12-31 22:00:00,263,160


In [63]:
import sys
import subprocess

# Upgrade related dependencies
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "hsfs", "hopsworks", "protobuf"])

  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)


CompletedProcess(args=['/opt/anaconda3/bin/python3', '-m', 'pip', 'install', '--upgrade', 'hsfs', 'hopsworks', 'protobuf'], returncode=0)

In [64]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-03-04 18:05:12,526 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:05:12,528 INFO: Initializing external client
2025-03-04 18:05:12,528 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:05:13,115 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Successfully connected to Hopsworks project: CDA_Project_1


In [65]:
feature_store = project.get_feature_store()

In [66]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [67]:
from hsfs.feature_group import FeatureGroup

# Create the feature group
feature_group = feature_store.create_feature_group(
    name="time_series_hourly_feature_group",
    version=1,
    description="Feature group for hourly time series data",
    primary_key=["timestamp"],  # Adjust primary key according to your dataset
    online_enabled=True
)
print("Feature group created successfully!")

Feature group created successfully!


In [68]:
feature_group = feature_store.create_feature_group(
    name="time_series_hourly_feature_group",
    version=1,
    description="Feature group for hourly time series data",  # Use the new 'id' column as the primary key
    online_enabled=True   # Enable online serving if needed
)
print("Feature group created with supported primary key for online serving.")

Feature group created with supported primary key for online serving.


In [69]:
ts_data.rename(columns={
    'pickup_hour': 'pickup_hour',
    'pickup_location_id': 'pickup_location_id',
    'rides': 'rides'
}, inplace=True)

In [70]:
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'])
ts_data['pickup_location_id'] = ts_data['pickup_location_id'].astype(int)
ts_data['rides'] = ts_data['rides'].astype(int)

In [71]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time=['pickup_hour']
)

In [72]:
ts_data = ts_data.rename(columns={
    "pickup_hour": "timestamp",
    "rides": "ride_count",
    "pickup_location_id": "pickup_location"
})

In [73]:
# Rename the columns
ts_data.rename(columns={
    'timestamp': 'pickup_hour',
    'pickup_location': 'pickup_location_id',
    'ride_count': 'rides'
}, inplace=True)

# Select only the required columns
ts_data_selected = ts_data[['pickup_hour', 'pickup_location_id', 'rides']]

# Insert the selected columns into the feature group
feature_group.insert(ts_data_selected, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |█| Rows 2277600/2277600 | Elapsed Time: 00:58 | Re


2025-03-04 18:06:14,995 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1215641/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/CDA_Project_1/Resources/jobs/time_series_hourly_feature_group_1_offline_fg_materialization/config_1741071906253) to trigger the materialization job again.



(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [74]:
# ts_data = ts_data.rename(columns={
#     'pickup_location_id': 'id',
#     'pickup_hour': 'timestamp',
#     'rides': 'value'
# })

In [75]:
# ts_data['id'] = ts_data['id'].astype('int64')  # Convert to bigint
# ts_data['timestamp'] = pd.to_datetime(ts_data['timestamp'])  # Convert to timestamp
# ts_data['value'] = ts_data['value'].astype('float64')  # Convert to double

In [76]:
# feature_group.insert(ts_data, write_options={"wait_for_job": False})

In [77]:
# feature_group.read().head(10)

In [78]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)  
print(f"DataFrame size: {df_memory_mb:.2f} MB")

DataFrame size: 857.47 MB


In [79]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277600 entries, 0 to 2277599
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int64         
 2   rides               int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 52.1 MB
